In [1]:
using PCT

Precompiling PCT
  ✓ PCT
  1 dependency successfully precompiled in 1 seconds. 13 already precompiled.


## $x^{\dagger} A x$

In [5]:
f, _ = @pct (A::Her) -> pullback((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j))); f

(A) -> 
    𝒫((x) -> 
        ∑((i, j), (x(i)'⋅x(j)⋅A(i, j))))

In [10]:
df = vdiff(f; settings=symmetry_settings)

(A) -> 
    (x) -> 
        (_d) -> 
            ∑((j), (x(j)⋅A(_d, j)⋅2.0))

## Trace of Product

In [11]:
f, _ = @pct begin
    tr = (X::CM) -> sum(i, X(i, i))
    matmul = (A::CM, B::CM) -> (i::Z, j::Z) -> sum(n, A(i, n) * B(n, j))
    pullback((A::CM, B::CM) -> tr(matmul(A, B) + matmul(B, A)))
end; f

let 
    tr = (X) -> 
        ∑((i), X(i, i))
    matmul = (A, B) -> 
        (i, j) -> 
            ∑((n), (A(i, n)⋅B(n, j)))
    𝒫((A, B) -> 
        tr((matmul(A, B)+matmul(B, A))))
end

In [12]:
vdiff(eval_all(f); settings=symmetry_settings)

(A, B) -> 
    (_d, _d_1) -> 
        (B(_d_1, _d)'⋅2.0), (_d, _d_1) -> 
        (A(_d_1, _d)'⋅2.0)

## Conjugate Gradient

In [13]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> begin
    E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) + sum(i, x(i) * b(i))
    pullback((α::R, β::R) -> E((i::Z) -> x(i) + α * (r(i) + β * p(i))))
end; f

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i), (x(i)⋅b(i)))+∑((i, j), (x(i)⋅x(j)⋅A(i, j)⋅0.5)))
        𝒫((α, β) -> 
            E((i) -> 
                (x(i)+((r(i)+(p(i)⋅β))⋅α))))
    end

In [14]:
df = vdiff(f)

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i), (x(i)⋅b(i)))+∑((i, j), (x(i)⋅x(j)⋅A(i, j)⋅0.5)))
        (α, β) -> 
            ∑((i), (𝒫(E)((i) -> 
                (x(i)+((r(i)+(p(i)⋅β))⋅α)), 1)(i)⋅(r(i)+(p(i)⋅β)))), ∑((i), (𝒫(E)((i) -> 
                (x(i)+((r(i)+(p(i)⋅β))⋅α)), 1)(i)⋅p(i)⋅α))
    end

In [15]:
df = vdiff(eval_all(f); settings=symmetry_settings)

(A, r, p, b, x) -> 
    (α, β) -> 
        (∑((i), (b(i)⋅(r(i)+(p(i)⋅β))))+∑((i, j), (A(i, j)⋅(x(j)+((r(j)+(p(j)⋅β))⋅α))⋅(r(i)+(p(i)⋅β))))), (∑((i), (b(i)⋅p(i)⋅α))+∑((i, j), (p(j)⋅A(i, j)⋅(x(i)+((r(i)+(p(i)⋅β))⋅α))⋅α)))

## Lagrangian Mechanics

In [16]:
_, ctx = @pct begin
    @space X begin
        type = (RF, ) -> RF
        linear = true
    end
end; 

In [18]:
f, _ = @pct _ ctx (L::RO, D::X) -> pullback((r::RF) -> sum((t::R), L(r(t), D(r)(t)))); f

(L, D) -> 
    𝒫((r) -> 
        ∑((t), L(r(t), D(r)(t))))

In [19]:
vdiff(f; settings=custom_settings(:clench_sum=>true))

(L, D) -> 
    (r) -> 
        ((_d) -> 
            𝒫((_z_2) -> 
                L(_z_2, D(r)(_d)))(r(_d), 1)+D'((_d) -> 
            𝒫((_z_1) -> 
                L(r(_d), _z_1))(D(r)(_d), 1)))

## Hartree Fock Gradient and Hessian Vector Product

In [20]:
_, ctx = @pct begin    
    @space T begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))
    end
end;

In [22]:
f, _ = @pct _ ctx (J::T) -> pullback((C::CM) -> sum((i, j, p, q, r, s),
    C(p, i)' * C(q, i) * C(r, j)' * C(s, j) * J(p, q, r, s))); f

(J) -> 
    𝒫((C) -> 
        ∑((i, j, p, q, r, s), (C(p, i)'⋅C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i))))

In [23]:
df = vdiff(f; settings=symmetry_settings)

(J) -> 
    (C) -> 
        (_d, _d_1) -> 
            ∑((j, p, r, s), (C(s, j)'⋅J(_d, p, s, r)⋅C(p, _d_1)⋅C(r, j)⋅4.0))

In [24]:
df |> get_body |> decompose |> pp |> simplify |> first

(C, _k) -> 
    (_a_2, _a_3) -> 
        (∑((_d, _d_1, p, s), (J(_d, p, s, _a_2)'⋅C(p, _d_1)'⋅_k(_d, _d_1)⋅C(s, _a_3)⋅4.0))+∑((_d, _d_1, p, r), (_k(_d, _d_1)'⋅J(_d, p, _a_2, r)⋅C(p, _d_1)⋅C(r, _a_3)⋅4.0))+∑((_d, j, r, s), (J(_d, _a_2, s, r)'⋅C(r, j)'⋅_k(_d, _a_3)⋅C(s, j)⋅4.0)))

## Foster Boys

In [25]:
f, _ = @pct (R::Her, R2::Her) -> pullback((C::CM) -> begin 
        r = (i::Z) -> sum((p, q), R(p, q) * C(i, p)' * C(i, q))
        sum(i, sum((p, q), R2(p, q) * C(i, p)' * C(i, q)) - r(i)' * r(i))
    end); f

(R, R2) -> 
    𝒫((C) -> 
        let 
            r = (i) -> 
                ∑((p, q), (C(i, p)'⋅R(p, q)⋅C(i, q)))
            ∑((i), (∑((p, q), (C(i, p)'⋅R2(p, q)⋅C(i, q)))+(r(i)'⋅r(i)⋅-1.0)))
        end)

In [26]:
df = vdiff(eval_all(f); settings=symmetry_settings)

(R, R2) -> 
    (C) -> 
        (_d, _d_1) -> 
            ((∑((p), (R2(_d_1, p)⋅C(_d, p)))+∑((_i, q, p), (C(_d, _i)'⋅R(_d_1, q)⋅R(_i, p)⋅C(_d, q)⋅C(_d, p)⋅-2.0)))⋅2.0)

## Localized Wannier Functions

In [27]:
_, ctx = @pct begin
    @domain BZ begin
        base = I
        periodic = true
    end
    
    @domain X begin
        symmetric = true
        contractable = false
    end
    
    @space Mmn begin
        type = (Z, Z, BZ, BZ) -> C
        symmetries = (((2, 1, 4, 3), :conj),)
    end

    @space SV begin
        type = (I,) -> C
        symmetries = (((1,), :ineg),)
    end
    
    @space Gauge begin
        type = (Z, Z, BZ) -> C
    end
end;

In [ ]:
f, _ = @pct _ ctx (S::Mmn, w::SV) -> pullback((U::Gauge) -> 
    begin
        ρ = (n::Z, b::X) -> sum((k::BZ, p, q), U(p, n, k)' * S(p, q, k, k + b) * U(q, n, k + b))
        sum((n, b::X), ρ(n, b)' * ρ(n, b))
    end); f

In [20]:
# df = vdiff(eval_all(f); settings=custom_settings(:logging=>true; preset = symmetry_settings))
df = vdiff(eval_all(f); settings=symmetry_settings)

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            ∑((b, p, k, _i, q), (4.0⋅U(q, _d_1, k)⋅U(p, _d_1, (b+_d_2))⋅S(_d, p, _d_2, (b+_d_2))⋅S(_i, q, (b+k), k)⋅U(_i, _d_1, (b+k))'))

In [48]:
f, _ = @pct begin
    x = 1
    y = (m::R) -> m - 2
    (j::I) ->  x+ y(j)
end; f

let 
    x = 1
    y = (m) -> 
        (m+-2.0)
    (j) -> 
        (y(j)+x)
end

In [26]:
f, _ = @pct begin
    y = (m::R) -> m - 2
    (j::I ∈ (1, 2)) -> y(j)
end; f

let 
    y = (m) -> 
        (m+-2.0)
    (j ∈ (1, 2)) -> 
        y(j)
end